Dataset Visulization

범주형, 수치형 변수 데이터 분석

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
import numpy as np

plt.rcParams['font.family'] = 'NanumGothic'

# 데이터 로드conda env export > myconda.yaml
file_path = 'train.csv'
spec_file_path = '데이터 명세_수정.xlsx'

df = pd.read_csv(file_path)
spec_df = pd.read_excel(spec_file_path)

# 범주형 & 수치형 변수 리스트 추출
categorical_columns = spec_df[spec_df["범주형 여부"] == 1]["컬럼명"].tolist()
numerical_columns = spec_df[spec_df["범주형 여부"] == 0]["컬럼명"].tolist()

print(f"✅ 범주형 변수 개수: {len(categorical_columns)}")
print(f"✅ 수치형 변수 개수: {len(numerical_columns)}")

# 저장할 폴더 생성
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
base_dir = f"results_{current_time}"
boxplot_dir = os.path.join(base_dir, "boxplots")  # Boxplot 저장 폴더
category_plot_dir = os.path.join(base_dir, "category_plots")  # 범주형 변수 그래프 저장 폴더
os.makedirs(boxplot_dir, exist_ok=True)
os.makedirs(category_plot_dir, exist_ok=True)

# 🔹 히트맵 (Heatmap) 추가 🔹
plt.figure(figsize=(12, 8))
correlation_matrix = df[numerical_columns].corr()  # 수치형 변수 간 상관관계 계산
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", linewidths=0.5)
plt.title("Feature Correlation Heatmap")
heatmap_path = os.path.join(base_dir, "heatmap.png")
plt.savefig(heatmap_path)
plt.close()

print(f"✅ 히트맵 저장 완료: {heatmap_path}")
# 🔹 결측치 탐색 🔹
missing_values = df.isnull().sum().reset_index()
missing_values.columns = ["컬럼", "결측치 개수"]
missing_values["결측 비율 (%)"] = (missing_values["결측치 개수"] / len(df)) * 100
missing_values = missing_values[missing_values["결측치 개수"] > 0]  # 결측치가 있는 변수만 저장

if not missing_values.empty:
    print("\n✅ 결측치 정보:")
    print(missing_values)

# 🔹 범주형 변수 분석 🔹
categorical_data = {}

for col in categorical_columns:
    cat_counts = df[col].value_counts(dropna=True).reset_index()
    cat_counts.columns = [col, "Count"]
    categorical_data[col] = cat_counts

    # 시각화 (barplot)
    plt.figure(figsize=(8, 4))
    sns.countplot(x=df[col], hue=df['임신 성공 여부'], hue_order=df['임신 성공 여부'].unique(), dodge=True)
    plt.title(f"{col} 분포 및 임신 성공 여부")
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.xticks(rotation=45)

    # 그래프 저장 (category_plots 폴더)
    plot_path = os.path.join(category_plot_dir, f"{col}_category_analysis.png")
    plt.savefig(plot_path)
    plt.close()  # 메모리 절약

# 🔹 수치형 변수 분석 (이상치 포함) 🔹
num_analysis = df[numerical_columns].describe()

# 이상치 탐색 (IQR 방법)
outlier_counts = {}

for col in numerical_columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # 이상치 개수 계산
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outlier_counts[col] = len(outliers)

    # 이상치 분포 시각화 (Boxplot)
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=df[col])
    plt.title(f"{col} - Boxplot (Outlier Detection)")
    plt.xlabel(col)

    # 그래프 저장 (boxplots 폴더)
    plot_path = os.path.join(boxplot_dir, f"{col}_boxplot.png")
    plt.savefig(plot_path)
    plt.close()  # 메모리 절약

# 이상치 개수 DataFrame 생성
outlier_df = pd.DataFrame(list(outlier_counts.items()), columns=["Feature", "Outlier Count"])

# 🔹 데이터 저장 🔹
output_file = os.path.join(base_dir, "feature_analysis.xlsx")

with pd.ExcelWriter(output_file) as writer:
    if not missing_values.empty:
        missing_values.to_excel(writer, sheet_name="Missing Values", index=False)
    num_analysis.to_excel(writer, sheet_name="Numerical Summary")
    outlier_df.to_excel(writer, sheet_name="Outlier Count", index=False)

    # 범주형 변수별 빈도수 데이터를 개별 시트로 저장
    for col, cat_df in categorical_data.items():
        cat_df.to_excel(writer, sheet_name=f"{col}_Counts", index=False)

print(f"✅ 분석 결과가 저장되었습니다: {base_dir}")
print(f"✅ Boxplot 저장 폴더: {boxplot_dir}")
print(f"✅ 범주형 변수 그래프 저장 폴더: {category_plot_dir}")


✅ 범주형 변수 개수: 48
✅ 수치형 변수 개수: 20


c:\Users\PC\anaconda3\envs\LGAimers6\Lib\site-packages\seaborn\utils.py:61: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from font(s) NanumGothic.
  fig.canvas.draw()
C:\Users\PC\AppData\Local\Temp\ipykernel_39448\3288157541.py:38: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from font(s) NanumGothic.
  plt.savefig(heatmap_path)


✅ 히트맵 저장 완료: results_20250209_043454\heatmap.png

✅ 결측치 정보:
                       컬럼  결측치 개수  결측 비율 (%)
3   임신 시도 또는 마지막 임신 경과 연수  246981  96.344855
5                특정 시술 유형       2   0.000780
8             단일 배아 이식 여부    6291   2.454057
9        착상 전 유전 검사 사용 여부  253633  98.939735
10       착상 전 유전 진단 사용 여부    6291   2.454057
28            배아 생성 주요 이유    6291   2.454057
39              총 생성 배아 수    6291   2.454057
40             미세주입된 난자 수    6291   2.454057
41        미세주입에서 생성된 배아 수    6291   2.454057
42               이식된 배아 수    6291   2.454057
43           미세주입 배아 이식 수    6291   2.454057
44               저장된 배아 수    6291   2.454057
45        미세주입 후 저장된 배아 수    6291   2.454057
46               해동된 배아 수    6291   2.454057
47                해동 난자 수    6291   2.454057
48            수집된 신선 난자 수    6291   2.454057
49            저장된 신선 난자 수    6291   2.454057
50               혼합된 난자 수    6291   2.454057
51       파트너 정자와 혼합된 난자 수    6291   2.454057
52       기증자 정자와 혼합된 난자 수    6291   2.45

C:\Users\PC\AppData\Local\Temp\ipykernel_39448\3288157541.py:98: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from font(s) NanumGothic.
  plt.savefig(plot_path)


✅ 분석 결과가 저장되었습니다: results_20250209_043454
✅ Boxplot 저장 폴더: results_20250209_043454\boxplots
✅ 범주형 변수 그래프 저장 폴더: results_20250209_043454\category_plots


Feature selection dataset 생성

In [10]:
import pandas as pd

# 🔹 Feature Selection: 결측치 25% 이상 컬럼 제거 및 "특정 시술 유형" 제외 🔹

# 결측치 25% 이상 컬럼 제거
missing_threshold = 25.0  # 25% 이상 결측치 제거 기준
missing_values = df.isnull().sum() / len(df) * 100  # 결측치 비율 계산
drop_cols = missing_values[missing_values > missing_threshold].index.tolist()

print(f"✅ 제거할 컬럼 (결측치 25% 이상): {drop_cols}")

# "특정 시술 유형" 컬럼 제외
if "배란 유도 유형" in df.columns:
    drop_cols.append("배란 유도 유형")
    print("✅ '배란 유도 유형' 컬럼 제외됨")

# 데이터셋에서 해당 컬럼 제거
df_selected = df.drop(columns=drop_cols)

# Feature Selection 후 데이터셋 저장 (UTF-8-SIG 인코딩 적용)
selected_output_file = os.path.join(base_dir, "selected_features.csv")
df_selected.to_csv(selected_output_file, index=False, encoding="utf-8-sig")


print(f"✅ Feature Selection 완료: {selected_output_file}")
print(f"✅ 최종 사용된 컬럼 개수: {len(df_selected.columns)}")
print(f"✅ 최종 컬럼 목록: {df_selected.columns.tolist()}")


✅ 제거할 컬럼 (결측치 25% 이상): ['임신 시도 또는 마지막 임신 경과 연수', '착상 전 유전 검사 사용 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자 해동 경과일', '배아 해동 경과일']
✅ '배란 유도 유형' 컬럼 제외됨
✅ Feature Selection 완료: results_20250209_043454\selected_features.csv
✅ 최종 사용된 컬럼 개수: 62
✅ 최종 컬럼 목록: ['ID', '시술 시기 코드', '시술 당시 나이', '시술 유형', '특정 시술 유형', '배란 자극 여부', '단일 배아 이식 여부', '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 여성 요인', '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '배아 생성 주요 이유', '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '난자 출처', '정자 출처', '난자 기증자 나이', '정자

Feature selection with 불임 이유 컬럼

In [12]:
import pandas as pd
import os

# 🔹 Feature Selection: 결측치 25% 이상 컬럼 제거 및 "배란 유도 유형" 제외 🔹

# 결측치 25% 이상 컬럼 제거
missing_threshold = 25.0  # 25% 이상 결측치 제거 기준
missing_values = df.isnull().sum() / len(df) * 100  # 결측치 비율 계산
drop_cols = missing_values[missing_values > missing_threshold].index.tolist()

print(f"✅ 제거할 컬럼 (결측치 25% 이상): {drop_cols}")

# "배란 유도 유형" 컬럼 제외
if "배란 유도 유형" in df.columns:
    drop_cols.append("배란 유도 유형")
    print("✅ '배란 유도 유형' 컬럼 제외됨")

# 데이터셋에서 해당 컬럼 제거
df_selected = df.drop(columns=drop_cols)

# 🔹 불임 원인 카테고리 통합 🔹
infertility_cols = [col for col in df.columns if "불임 원인" in col]  # 불임 원인 관련 컬럼 찾기
df_selected["불임 원인 존재"] = df[infertility_cols].max(axis=1)  # 하나라도 1이면 1

df_selected = df_selected.drop(columns=infertility_cols)  # 기존 불임 원인 컬럼 제거

# 🔹 Feature Selection 후 데이터셋 저장 🔹
selected_output_file = os.path.join(base_dir, "selected_features_with_infertility.csv")
df_selected.to_csv(selected_output_file, index=False, encoding="utf-8-sig")

print(f"✅ Feature Selection 완료: {selected_output_file}")
print(f"✅ 최종 사용된 컬럼 개수: {len(df_selected.columns)}")
print(f"✅ 최종 컬럼 목록: {df_selected.columns.tolist()}") 

✅ 제거할 컬럼 (결측치 25% 이상): ['임신 시도 또는 마지막 임신 경과 연수', '착상 전 유전 검사 사용 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자 해동 경과일', '배아 해동 경과일']
✅ '배란 유도 유형' 컬럼 제외됨
✅ Feature Selection 완료: results_20250209_043454\selected_features_with_infertility.csv
✅ 최종 사용된 컬럼 개수: 46
✅ 최종 컬럼 목록: ['ID', '시술 시기 코드', '시술 당시 나이', '시술 유형', '특정 시술 유형', '배란 자극 여부', '단일 배아 이식 여부', '착상 전 유전 진단 사용 여부', '배아 생성 주요 이유', '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', '난자 채취 경과일', '난자 혼합 경과일', '배아 이식 경과일', '임신 성공 여부', '불임 원인 존재']
